# Connect with Manufacturers for remote SMC

In [1]:
%%javascript
document.title="City Agent"

<IPython.core.display.Javascript object>

#### Imports

In [2]:
from aries_cloudcontroller import AriesAgentController
import os
from pprintpp import pprint
from termcolor import colored
import time

In [3]:
# If the identifiers are not stored for some reason
try:
    # Load variables – assuming they have been written to the store in a previous notebook
    print("--- LOAD VARIABLES FROM JUPYTERLAB STORE ---")
    
    %store -r schema_manufacturer_id
    %store -r cred_def_manufacturer_id
    %store -r authority_did
    
    print(f"schema_manufacturer_id: {schema_manufacturer_id}")
    print(f"cred_def_manufacturer_id: {cred_def_manufacturer_id}")
    print(f"authority_did: {authority_did}")
    
except Exception as e:
    
    print("--- DEFINE VARIABLES FROM MANUALLY COPIED identifiers DICT ---")
    
    # Copy an updated version if necessary
    identifiers = {
        'city_schema_identifiers': {
            'cred_def': 'BQmsjXtsjzzmTtdwQg5Ftn:3:CL:239757:default',
            'schema_id': 'BQmsjXtsjzzmTtdwQg5Ftn:2:certify-city-agency:0.0.1',
        },
        'manufacturer_schema_identifiers': {
            'cred_def': 'BQmsjXtsjzzmTtdwQg5Ftn:3:CL:239761:default',
            'schema_id': 'BQmsjXtsjzzmTtdwQg5Ftn:2:certify-manufacturer:0.0.1',
        },
        'authority-did': 'KxYXF6vfTXMoBNg2uGt79g'
    }

    # Get manufacturer schema identifiers
    schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
    cred_def_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["cred_def"]
    
    # Get authority did
    authority_did = identifiers['authority-did']

    # Load variables to Jupyter store
    %store schema_manufacturer_id
    %store cred_def_manufacturer_id
    %store authority_did

--- LOAD VARIABLES FROM JUPYTERLAB STORE ---
schema_manufacturer_id: BQmsjXtsjzzmTtdwQg5Ftn:2:certify-manufacturer:0.0.1
cred_def_manufacturer_id: BQmsjXtsjzzmTtdwQg5Ftn:3:CL:239761:default
authority_did: KxYXF6vfTXMoBNg2uGt79g


#### Initialize `City` Agent Controller

In [4]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://city-agent:3021 and an api key of adminApiKey


## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [5]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

In [6]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

        
def verifier_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    

    if state == "request_sent":
        print("Presentation Request\n")
        print(payload["presentation_request"])
        print("\nThe presentation request is encoded in base64 and packaged into a DIDComm Message\n")
        print(payload["presentation_request_dict"])
        print("\nNote the type defines the protocol present-proof and the message request-presentation\n")
        print(payload["presentation_request_dict"]["@type"])
    elif state == "presentation_received":
        print("Presentation Received")
        print("We will not go into detail on this payload as it is comparable to the presentation_sent we looked at in the earlier cell.")
        print("This is the full payload\n")
        print(payload)
    else:
        print("Paload \n")
        print(payload)

In [7]:
# Init listener list
listeners = []

# Add listeners defined in previous cell
connection_listener = {"handler": connections_handler, "topic": "connections"}
listeners.append(connection_listener)

verifier_listener = {"topic": "present_proof", "handler": verifier_proof_handler}
listeners.append(verifier_listener)

# Register listeners
agent_controller.register_listeners(listeners)

## Define Presentation Request Object

The below cell defines a generic presentation request object, that can be sent across specific connections requesting that they produce a presentation containing the identified attributes and meeting the restrictions.

It is often useful to define your request objects first, then reuse these objects across many connections you wish to request a proof from. 

Duplicate and customise the below cell as many times as you need. It may be useful to save these request objects either to the jupyter store using %store or through

TODO: Detail the full set of restrictions available to a verifier.

In [8]:
# We add a constraint that the attribute must originate from this schema

# Define the list of attributes and restrictions under which each attribute was issued that a prover must satisfy with a presentation
# NOTE: if identifying a schema or credential definition then the attribute name must be contained within the corresponding schema.
req_attrs = [
    {"name": "isManufacturer", "restrictions": [{"schema_id": schema_manufacturer_id, "issuer_did": authority_did}]},
]

# We could extend this to request the name attribute aswell if we wanted.


manufacturer_proof_request = {
    "name": "Name of Proof Request",
    "version": "1.0",
    "requested_attributes": {
        # They generally follow this uuid pattern. Unique identifier for attribute within context of this proof request
        # Note that req_attr['name'] gets the attribute name of each object. E.g. domain and name in this case
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    # Predicates allow us to specify range proofs or set membership on attributes. For example greater than 10.
    # We will ignore these for now.
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
    # You can also request the entire proof request be non-revoked
    "non_revoked":  {"to": int(time.time())}
}

## Create Invitation

Note the current arguments specified are in their default configurations. 

In [9]:
# Alias for invited connection
alias = None
auto_accept = "true"
# Use public DID?
public = "false"
# Should this invitation be usable by multiple invitees?
multi_use = "true"

invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)
# Is equivalent to above. Arguments are optionally
# invitation_response = await agent_controller.connections.create_invitation()

# You probably want to keep this somewhere so you can enage in other protocols with this connection.
connection_id = invitation_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  4f86a101-3e93-4fa2-817f-0b60ebe9d61b
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
invitation


## Share Invitation Object with External Agent

Typically in this jupyter notebook playground that involves copying it across to another agent's business logic notebook where they are the invitee. (see prover_template)

In [10]:
invitation = invitation_response["invitation"]
## Copy this output
pprint(invitation)

{
    '@id': '808061b1-42a3-495b-b8c8-c9c15e9c6e05',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['3XPvTHWspTMf1wWWtsaFeviaEPaejD5c1Z3XRwRys5bA'],
    'serviceEndpoint': 'https://54a5a5d5a57e.ngrok.io',
}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  675d8426-13b6-4257-bf6b-ad6bbec53215
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  675d8426-13b6-4257-bf6b-ad6bbec53215
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  675d8426-13b6-4257-bf6b-ad6bbec53215
State :  respon

## Optional: Recieve Proof Proposal

A Verifier is not the only role that can initiate the present-proof protocol. A holder can also send a proposal for a presentation that they can provide.

In [ ]:
# TODO...

## Send Proof Request

This identifies a connection and a proof request object (you will need to update the variable name enclosed with <>) 

In [13]:
proof_request = {
    "comment": "<some optional comment>",
    "connection_id": "675d8426-13b6-4257-bf6b-ad6bbec53215",
    "proof_request": manufacturer_proof_request,
    # Do you want your agent to trace this request (for debugging)
    "trace": False
}

proof_request_response = await agent_controller.proofs.send_request(proof_request)


---------------------------------------------------------------------

Handle present-proof
Connection ID :  675d8426-13b6-4257-bf6b-ad6bbec53215
Presentation Exchange ID :  fcd73940-f152-4bd4-ac9d-da67d81ddf4d
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'nonce': '471517040101429148699515', 'name': 'Name of Proof Request', 'version': '1.0', 'requested_attributes': {'0_isManufacturer_uuid': {'name': 'isManufacturer', 'restrictions': [{'schema_id': 'BQmsjXtsjzzmTtdwQg5Ftn:2:certify-manufacturer:0.0.1', 'issuer_did': 'KxYXF6vfTXMoBNg2uGt79g'}]}}, 'requested_predicates': {}, 'non_revoked': {'to': 1628149482}}

The presentation request is encoded in base64 and packaged into a DIDComm Message

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '83bbb9db-0e62-42a3-aabe-9ec961154850', 'request_presentations~attach': [{'@id': 'li

## Get Presentation Exchange Record

This record keeps track of the current state of the presentation protocol, which must be in the `presentation_received` state before the presentation can be verified.

Note: This could also happen in the webhook logic.

In [ ]:
presentation_exchange_id = proof_request_response["presentation_exchange_id"]

pres_record = await agent_controller.proofs.get_record_by_id(presentation_exchange_id)

print(pres_record)

## Verify Presentation

Only if it is in the right state. 

Note: Verifying a presentation moves the state to `verified` regardless of whether the presentation request has been satisfied. To check this you must refer to the `verified` property on the response.

In [ ]:
verified_response = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

verified = verified_response["verified"]

## Parsing Disclosed Attribute Values from Presentation

A presentation object contains three classes of attributes. 
* Revealed Attributes: Attributes that were signed by an issuer and have been revealed in the presentation process
* Self Attested Attributes: Attributes that the prover has self attested to in the presentation object.
* Predicate proofs: Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

### Parse Revealed Attributes

In [ ]:
for (name, val) in verified_response['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print("\nAttribute : ", val)
    
    attr_name = verified_response["presentation_request"]["requested_attributes"][name]["name"]
    print("Attribute Name :  Raw Value")
    print(f"{attr_name}   :  {val['raw']}")

### Parse Self-Attested Attributes

In [ ]:
for (name, val) in verify['presentation']['requested_proof']['self_attested_attrs'].items():
    print(name)
    ## Slightly different for self attested attrs
    print(val)

## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [ ]:
## Custom Logic

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [16]:
await agent_controller.terminate()